In [10]:
import pandas as pd
import numpy as np


from sklearn.cluster import KMeans, DBSCAN
# from sklearn.neighbors import NearestNeighbors

# removing outliers
from scipy import stats

import seaborn as sns
import matplotlib.pyplot as plt
# from matplotlib import cm
# from mpl_toolkits.mplot3d import axes3d, Axes3D, proj3d
# from mpl_toolkits.mplot3d.art3d import Poly3DCollection


#### Display options

%matplotlib inline
# %pylab inline
# %matplotlib qt
# %matplotlib notebook

%config InlineBackend.figure_format = 'retina'


# Jupytper Config
# %config Completer.use_jedi = False
# %config IPCompleter.greedy = True
# pd.set_option('display.max_columns', None)
# pd.options.display.max_rows = 30
# pd.set_option('display.max_colwidth', None)

pd.options.mode.chained_assignment = 'raise'


# Import External Files

In [11]:
# Data file produced by tests.
df = pd.read_csv('./CSV/colour_data_classified.csv',index_col='Unnamed: 0')

# Colour Pallets used for testing
# pallets = pd.read_csv('./CSV/pallets_diction_ary.csv',index_col=0)

# Prevents CSV Tuples being imported as Strings via ast — Abstract Syntax Trees
import ast

df.ishihara_list = df.ishihara_list.apply(ast.literal_eval)
df.COLORS_ON = df.COLORS_ON.apply(ast.literal_eval)
df.COLORS_OFF = df.COLORS_OFF.apply(ast.literal_eval)

In [12]:
# df.head().append(df.head(-5))
df

,user,correct,recorded_result,mask_image,cb_type1,cb_type2,ncb,datetime,random_spread,pallet_used,...,COLORS_ON,COLORS_OFF,KMeans,DBSCAN,AgglomerativeClustering,cluster_classification,AdaBoostClassifier,BernoulliNB,SVC,MLPClassifier
0,2NuSkbN87vtA3NUnHnk7MA,1,5,5,0,0,1,16/6/20 11:22,10,bear,...,"[(220, 105, 53), (239, 192, 86), (227, 162, 61...","[(71, 138, 38), (92, 73, 38), (153, 165, 49), ...",0,0,1,1,0,0,0,0
1,2NuSkbN87vtA3NUnHnk7MA,1,C,C,0,0,1,16/6/20 11:20,10,bear,...,"[(232, 116, 65), (247, 198, 95), (221, 152, 56...","[(65, 149, 38), (99, 72, 41), (168, 165, 50), ...",0,0,1,1,0,0,0,0
2,2NuSkbN87vtA3NUnHnk7MA,1,D,D,0,0,1,16/6/20 11:30,10,bear,...,"[(226, 114, 54), (253, 186, 87), (233, 148, 70...","[(75, 142, 40), (96, 64, 34), (162, 152, 41), ...",0,0,1,1,0,0,0,0
3,2NuSkbN87vtA3NUnHnk7MA,1,N,N,0,0,1,16/6/20 11:21,10,bear,...,"[(231, 116, 53), (253, 203, 81), (215, 149, 67...","[(70, 149, 26), (96, 73, 48), (163, 158, 57), ...",0,0,1,1,0,0,0,0
4,2NuSkbN87vtA3NUnHnk7MA,0,N,E,0,0,1,16/6/20 11:22,10,bear,...,"[(223, 102, 47), (244, 188, 85), (217, 147, 55...","[(63, 147, 29), (103, 82, 46), (155, 153, 53),...",0,0,1,1,0,3,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5816,BD,1,C,C,0,1,1,22:28.0,10,standard_3,...,"[(134, 182, 116), (113, 155, 97), (189, 203, 1...","[(240, 154, 32), (200, 119, 101), (234, 139, 9...",0,0,1,1,0,1,0,0
5817,BD,1,D,D,0,1,1,23:02.2,10,standard_3,...,"[(135, 185, 107), (127, 171, 89), (190, 203, 1...","[(237, 152, 45), (203, 112, 99), (235, 142, 11...",0,0,1,1,0,1,0,0
5818,BD,1,2,2,0,1,1,24:26.4,10,standard_3,...,"[(135, 178, 121), (114, 159, 90), (186, 198, 1...","[(244, 152, 36), (198, 126, 94), (229, 132, 10...",0,0,1,1,0,1,0,0
5819,BD,1,5,5,0,1,1,25:41.5,10,standard_3,...,"[(131, 179, 105), (114, 163, 100), (173, 207, ...","[(235, 157, 39), (195, 119, 86), (230, 146, 97...",0,0,1,1,0,1,0,0


### Each time a colourblind test is used a pallet is chosen.  
It's made up of 12 colours (in hex), that are then randomised a little.  
1-6 go 'on_' the symbol and 7-12 fill in the background.  

[#e26f39,#f6c458,#e09c3e,#e69e40,#ed7b3d,#f5be67,#469224,#604929,#a09f33,#dfd548,#89bcb2,#dccb47]

### RGB Values Returned

Two columns are of interest are returned from each test,   
each row con_taining a list of 6 colours with Red/Green/Blue Values  

COLORS_ON : [(220,105,53),(239,192,86),(227,162,61),(222,153,60),(235,113,53),(240,191,102)]  
COLORS_OFF : [(71,138,38),(92,73,38),(153,165,49),(221,221,70),(144,190,182),(224,196,63)]  
  
I need to isolate the colours into groups based on_ the colours.  

A single tuple is a colour, made up of (Red, Green, Blue) values.  

I need to 
[(a,b,c),(d,e,f) ...  ]  
into: ab,bc,ca, de,ef,fd ... etc  



## First: Filter the Users by response
- Take a selection of respon_ses from the entire data pool that have the 'most correct' answers for that particular set of colours.  
- The 'best' answers (most failed in most cases) are used to build an adjusted set of colours to test in the next round.

## For Each Pallet Group the values

- Now that the best values are grouped.
- The goal here is (red/green) (green/blue) (blue/red) tuples, for each colour, in each pallet, of every respon_se.
- These are eventually fed into DBSCAN clustering to shave off the outliers.
- Then we use 3D kmeans clustering, which finds a mean-center of our best batch of respon_ses.
- This becomes the base colour for our new pallet to test people with.


In [66]:
def quickplot(x,y):
    return sns.jointplot(x, y, hue=)
#     sns.jointplot(data=penguins, x="bill_length_mm", y="bill_depth_mm", hue="species")
#     Need to include all three sets of vectors into 1 chart as hue

    
    
    
    
    

# # ploting the 3d space.
# def plot_3d(x,y):
# #     scatter on is all of the colours as points in one list.
#     scatter_on =  colour_1 + group_on2 + group_on3 + group_on4 + group_on5 + group_on6
#     scatter_off =  group_off1 + group_off2 + group_off3 + group_off4 + group_off5 + group_off6

# #     This pulls the red,green,blue values for each point on the symbol
#     xon_red = [i[0] for i in scatter_on]
#     yon_green = [i[1] for i in scatter_on]
#     zon_blue = [i[2] for i in scatter_on]

#     #     This pulls the red,green,blue values for each point in the background
#     xoff_red = [i[0] for i in scatter_off]
#     yoff_green = [i[1] for i in scatter_off]
#     zoff_blue = [i[2] for i in scatter_off]

# ploting the 3d space.
# def colourspace_plot(pallet):
#     fig = plt.figure(figsize=(12,12))
#     ax = fig.add_subplot(111, projection='3d')
#     plt.rcParams['legend.fontsize'] = 10 

#     ax.scatter(xon_red, yon_green, zon_blue, marker='x', alpha=0.7, label='rgb_on')
#     ax.scatter(xoff_red, yoff_green, zoff_blue, marker='o', alpha=0.4, label='rgb_off')

#     ax.set_xlabel('Red')
#     ax.set_ylabel('Green')
#     ax.set_zlabel('Blue')
#     ax.legend(loc='lower right')

#     plt.show()

In [148]:
CALIBRATION_GROUP = pd.DataFrame()

for pallet in df['pallet_used'].unique():

    df_pallet = df[df['pallet_used'] == pallet] # iterate for the pallet.

    # Set for type1 colourblind correct
    df1 = df_pallet[(df_pallet['cb_type1'] == 1) & (df_pallet['cluster_classification'] == 2) & (df_pallet['correct'] == 1)] 
    df2 = df_pallet[(df_pallet['cb_type1'] == 1) & (df_pallet['cluster_classification'] == 4) & (df_pallet['correct'] == 1)]
    df3 = df_pallet[(df_pallet['cb_type1'] == 1) & (df_pallet['cluster_classification'] == 1) & (df_pallet['correct'] == 0)]

    # Set for type2 colourblind correct
    df4 = df_pallet[(df_pallet['cb_type2'] == 1) & (df_pallet['cluster_classification'] == 2) & (df_pallet['correct'] == 1)] 
    df5 = df_pallet[(df_pallet['cb_type2'] == 1) & (df_pallet['cluster_classification'] == 4) & (df_pallet['correct'] == 1)]
    df6 = df_pallet[(df_pallet['cb_type2'] == 1) & (df_pallet['cluster_classification'] == 1) & (df_pallet['correct'] == 0)]

    # Set for ncb correct
    df7 = df_pallet[(df_pallet['ncb'] == 1) & (df_pallet['cluster_classification'] == 2) & (df_pallet['correct'] == 0)] 
    df8 = df_pallet[(df_pallet['ncb'] == 1) & (df_pallet['cluster_classification'] == 4) & (df_pallet['correct'] == 0)]
    df9 = df_pallet[(df_pallet['ncb'] == 1) & (df_pallet['cluster_classification'] == 1) & (df_pallet['correct'] == 1)] 

    calibration = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9],axis=0) #     This pulls together the criteria for improvement.
    calibration = calibration[['pallet_used','ishihara_list']]

    CALIBRATION_GROUP = pd.concat([CALIBRATION_GROUP,calibration],axis=0)

CALIBRATION_GROUP = CALIBRATION_GROUP.reset_index()
print(f'Dataset ready for calibration')

# Globals
NEW_COLOURS_LIST = pd.DataFrame()
PALLET_NAMES = calibration['pallet_used'].unique()
CALIBRATION_REPORT = {}

# Create the calibration Pallet.
def create_calibration_pallet(CALIBRATION_GROUP):
    for pallet in CALIBRATION_GROUP['pallet_used'].unique(): # this cycles for each pallet used in the data individually

        df_pallet = CALIBRATION_GROUP[CALIBRATION_GROUP['pallet_used'] == pallet] # iterate for the pallet.

        if len(df_pallet['pallet_used']) > 30: # check there are enough samples
            print(f"{pallet} has {len(df_pallet['pallet_used'])} suitable responses for calibration.")
        else:
            print(f"WARNING: for {pallet} we have less than 30 samples. Skipping")
            continue

        # Dictionary for iteration, made up of all colours.
        colours = {
            'colour_1' : {
                'red' : [colour[0][0] for colour in df_pallet['ishihara_list']],
                'green': [colour[0][1] for colour in df_pallet['ishihara_list']],
                'blue' : [colour[0][2] for colour in df_pallet['ishihara_list']],
            },
            'colour_2' : {
                'red' : [colour[1][0] for colour in df_pallet['ishihara_list']],
                'green': [colour[1][1] for colour in df_pallet['ishihara_list']],
                'blue' : [colour[1][2] for colour in df_pallet['ishihara_list']],
            },
            'colour_3' : {
                'red' : [colour[2][0] for colour in df_pallet['ishihara_list']],
                'green': [colour[2][1] for colour in df_pallet['ishihara_list']],
                'blue' : [colour[2][2] for colour in df_pallet['ishihara_list']],
            },
            'colour_4' :{
                'red' : [colour[3][0] for colour in df_pallet['ishihara_list']],
                'green': [colour[3][1] for colour in df_pallet['ishihara_list']],
                'blue' : [colour[3][2] for colour in df_pallet['ishihara_list']],
            },
            'colour_5' :{
                'red' : [colour[4][0] for colour in df_pallet['ishihara_list']],
                'green': [colour[4][1] for colour in df_pallet['ishihara_list']],
                'blue' : [colour[4][2] for colour in df_pallet['ishihara_list']],
            },
            'colour_6' :{
                'red' : [colour[5][0] for colour in df_pallet['ishihara_list']],
                'green': [colour[5][1] for colour in df_pallet['ishihara_list']],
                'blue' : [colour[5][2] for colour in df_pallet['ishihara_list']],
            },
            'colour_7' :{
                'red' : [colour[6][0] for colour in df_pallet['ishihara_list']],
                'green': [colour[6][1] for colour in df_pallet['ishihara_list']],
                'blue' : [colour[6][2] for colour in df_pallet['ishihara_list']],
            },
            'colour_8' :{
                'red' : [colour[7][0] for colour in df_pallet['ishihara_list']],
                'green': [colour[7][1] for colour in df_pallet['ishihara_list']],
                'blue' : [colour[7][2] for colour in df_pallet['ishihara_list']],
            },
            'colour_9' :{
                'red' : [colour[8][0] for colour in df_pallet['ishihara_list']],
                'green': [colour[8][1] for colour in df_pallet['ishihara_list']],
                'blue' : [colour[8][2] for colour in df_pallet['ishihara_list']],
            },
            'colour_10' :{
                'red' : [colour[9][0] for colour in df_pallet['ishihara_list']],
                'green': [colour[9][1] for colour in df_pallet['ishihara_list']],
                'blue' : [colour[9][2] for colour in df_pallet['ishihara_list']],
            },
            'colour_11' :{
                'red' : [colour[10][0] for colour in df_pallet['ishihara_list']],
                'green': [colour[10][1] for colour in df_pallet['ishihara_list']],
                'blue' : [colour[10][2] for colour in df_pallet['ishihara_list']],
            },
            'colour_12' :{
                'red' : [colour[11][0] for colour in df_pallet['ishihara_list']],
                'green': [colour[11][1] for colour in df_pallet['ishihara_list']],
                'blue' : [colour[11][2] for colour in df_pallet['ishihara_list']],
            }
        }

#     Convert Dictionary to pandas DataFrame
    coloursdf = pd.DataFrame(colours)

#     Create List of Pairs For Mashing.
    calibrated_pallet = []
    
    pallet_report = {}
    
#     Pairing & Clustering
    for colour,rgb in coloursdf.items():

        red_green = pd.DataFrame({'red': rgb[0],'green': rgb[1]})
        green_blue = pd.DataFrame({'green': rgb[1],'blue': rgb[2]})
        blue_red = pd.DataFrame({'blue': rgb[2],'red': rgb[0]})
           
        calibrator = [red_green,green_blue,blue_red] # List of vectors between R,G,B  X,Y,Z points.

#         DBSCAN Variables.
        eps_val = 15 # EPS Max dist is 20, for spread 10. 10 = 50% of range
        min_samples_val = int(len(calibrator[0].iloc[:, 0]) * .75) # 90% of the total pool required to make 1 cluster.

#         DBSCAN
        cluster_centers = [] # list of pair centers.
        
        pair_names = [calibrator.columns]
        for pair in calibrator:
            status = {}
            pair_name = pair_namescalibrator.columns == pair
#             # prune outliers
#             status['primary_count'] = len(pair.iloc[:, 0]) # Get data len # consider using .shape[0]
#             pair = pair[(np.abs(stats.zscore(pair.iloc[:, 0])) < 3)] # uses the zscore function of df less than a SD of 3
#             pair = pair[(np.abs(stats.zscore(pair.iloc[:, 1])) < 3)] # uses the zscore function of df less than a SD of 3
#             status['secondary_count'] = len(pair.iloc[:, 0]) # Get data len
#             status['primary_removed'] = status['primary_count'] - status['secondary_count'] # Get dif  in data len
            
#             dbscan = DBSCAN(eps=eps_val, min_samples=min_samples_val, metric = 'euclidean').fit_predict(pair) # create a big cluster
#             pair['dbscan'] = dbscan # assign it as an array
#             dbscan = pair[pair['dbscan'] == 0] # Take cluster 0 
#             dbscan = dbscan.drop('dbscan',axis=1)
#             status['third_count'] = len(dbscan.iloc[:, 0]) # Get data len
#             status['secondary_removed'] = status['secondary_count'] - status['third_count'] # Get dif  in data len
            
# #             KMeans, finding new cluster centers.
#             kmeans = KMeans(n_clusters=1).fit(dbscan) # fit KMEANS
#             cluster_centers.append(kmeans.cluster_centers_) # Take the center value of the cluster.

# #             quickplot(dbscan)
            
#             pallet_report[str(pair)] = status
#         CALIBRATION_REPORT[pallet] = pallet_report

#         red_corrected = int((cluster_centers[0][0][0] + cluster_centers[2][0][1]) / 2) # red
#         green_corrected = int((cluster_centers[0][0][1] + cluster_centers[1][0][0]) / 2) # green
#         blue_corrected = int((cluster_centers[1][0][1] + cluster_centers[2][0][0]) / 2) # blue
        
#         calibrated_rgb=[red_corrected,green_corrected,blue_corrected]
#         calibrated_rgb = '#%02x%02x%02x' % (red_corrected, green_corrected, blue_corrected) # Turn Tuple to Hex
#         calibrated_pallet.append(calibrated_rgb) # add the list of colours to a pallet list
        
#     NEW_COLOURS_LIST[pallet] = calibrated_pallet

Dataset ready for calibration


In [149]:
create_calibration_pallet(CALIBRATION_GROUP)

bear has 211 suitable responses for calibration.
bluey2 has 171 suitable responses for calibration.
camo2 has 301 suitable responses for calibration.
greys has 281 suitable responses for calibration.
inverse_ring has 215 suitable responses for calibration.
orn_grn has 298 suitable responses for calibration.
pinks has 229 suitable responses for calibration.
reversed_ring has 217 suitable responses for calibration.
ring has 368 suitable responses for calibration.
standard_1 has 180 suitable responses for calibration.
standard_2 has 181 suitable responses for calibration.
standard_3 has 302 suitable responses for calibration.
     red  green
0    139    179
1    125    167
2    122    181
3    151    165
4    158    197
..   ...    ...
297  134    182
298  135    185
299  135    178
300  131    179
301  133    177

[302 rows x 2 columns]
     green  blue
0      179   110
1      167   122
2      181   109
3      165   117
4      197   135
..     ...   ...
297    182   116
298    185   107


In [150]:
NEW_COLOURS_LIST.T

""


In [147]:
pd.DataFrame(CALIBRATION_REPORT)

,standard_3
blue red dbscan\n0 0 250 0\n1 9 243 0\n2 1 252 0\n3 3 244 0\n5 11 243 0\n.. ... ... ...\n297 5 247 0\n298 5 253 0\n299 5 248 0\n300 1 249 0\n301 7 253 0\n\n[287 rows x 3 columns],"{'primary_count': 302, 'secondary_count': 287,..."
blue red dbscan\n0 4 254 0\n1 16 253 0\n2 19 249 0\n3 4 254 0\n4 21 240 0\n.. ... ... ...\n297 4 251 0\n298 1 248 0\n299 6 249 0\n300 4 244 0\n301 1 254 0\n\n[297 rows x 3 columns],"{'primary_count': 302, 'secondary_count': 297,..."
blue red dbscan\n0 36 240 0\n1 49 238 0\n2 38 251 0\n3 53 250 0\n5 39 246 0\n.. ... ... ...\n297 32 240 0\n298 45 237 0\n299 36 244 0\n300 39 235 0\n301 46 239 0\n\n[297 rows x 3 columns],"{'primary_count': 302, 'secondary_count': 297,..."
blue red dbscan\n0 89 198 0\n1 96 202 0\n2 103 208 0\n3 99 210 0\n5 96 199 0\n.. ... ... ...\n297 101 200 0\n298 99 203 0\n299 94 198 0\n300 86 195 0\n301 86 200 0\n\n[287 rows x 3 columns],"{'primary_count': 302, 'secondary_count': 287,..."
blue red dbscan\n0 90 201 0\n1 88 201 0\n2 94 209 0\n3 95 189 0\n5 80 203 0\n.. ... ... ...\n297 91 193 0\n298 98 209 0\n299 93 195 0\n300 90 206 0\n301 102 206 0\n\n[297 rows x 3 columns],"{'primary_count': 302, 'secondary_count': 297,..."
blue red dbscan\n0 95 122 0\n1 79 136 0\n2 88 126 0\n3 84 112 0\n5 99 108 0\n.. ... ... ...\n297 84 131 0\n298 91 131 0\n299 97 125 0\n300 91 113 0\n301 90 124 0\n\n[292 rows x 3 columns],"{'primary_count': 302, 'secondary_count': 292,..."
blue red dbscan\n0 97 125 0\n1 100 128 0\n2 106 124 0\n3 100 136 0\n4 110 140 -1\n.. ... ... ...\n297 97 113 0\n298 89 127 0\n299 90 114 0\n300 100 114 0\n301 98 131 0\n\n[302 rows x 3 columns],"{'primary_count': 302, 'secondary_count': 302,..."
blue red dbscan\n0 105 232 0\n1 101 235 0\n2 103 244 0\n3 119 227 0\n4 108 215 0\n.. ... ... ...\n297 98 234 0\n298 110 235 0\n299 109 229 0\n300 97 230 0\n301 112 226 0\n\n[297 rows x 3 columns],"{'primary_count': 302, 'secondary_count': 297,..."
blue red dbscan\n0 110 139 0\n1 122 125 0\n2 109 122 0\n3 117 151 0\n5 124 150 0\n.. ... ... ...\n297 116 134 0\n298 107 135 0\n299 121 135 0\n300 105 131 0\n301 104 133 0\n\n[292 rows x 3 columns],"{'primary_count': 302, 'secondary_count': 292,..."
blue red dbscan\n0 117 183 0\n1 118 196 0\n2 128 190 0\n3 123 196 0\n5 127 180 0\n.. ... ... ...\n297 119 189 0\n298 117 190 0\n299 116 186 0\n300 117 173 0\n301 107 179 0\n\n[297 rows x 3 columns],"{'primary_count': 302, 'secondary_count': 297,..."
